In [7]:
# set run environment (local/colab), if colab move proper dir
import os
import json
from pathlib import Path

if os.getenv("COLAB_RELEASE_TAG"):
    colab = True

    from google.colab import drive
    drive.mount('/content/drive')
    %cd /content/drive/Othercomputers/My computer/EQILLM/

    !pip install -r requirements.txt -q --exists-action i
    !pip install transformers[torch] -q --exists-action i
    !pip install accelerate -U -q --exists-action i
else:
    colab = False

import torch, gc
import wandb
import runpy
import pickle
# from numba import cuda

from dotenv import load_dotenv, dotenv_values
from huggingface_hub import login
from tqdm.auto import tqdm
from tqdm.notebook import tqdm_notebook
from transformers import EarlyStoppingCallback


from eqillm import yeelight_eow_notification, param_combinations_recursive

dotenv_config = dotenv_values('env')
yeelight_notify = dotenv_config['YEELIGHT_NOTIFY'] if (('YEELIGHT_NOTIFY' in dotenv_config) & (colab==False)) else False

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
login(token=dotenv_config['HF_TOKEN'])
wandb.login(key=dotenv_config['WANDB_TOKEN'])

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Jakub\.netrc


Token is valid (permission: read).
Your token has been saved to C:\Users\Jakub\.cache\huggingface\token
Login successful


True

In [11]:
# if something is passed as list, all possible combinations of provided parameters will be created. 
# To omit that behaviour for variable requiring multiple values (for example, target modules in peft or data splits
# pass them as tuple 

params_tested = {'data_path': [
                               'data/PolarIs-Pathos.xlsx',
                               # 'data/polish_pathos_translated.xlsx',
                              ],
                 # -----------------------
                 'wandb_init_params': {
                    'project': [
                                # "polish_pathos_translated",
                                "test"
                                 ],
                    'group':  "binary",
                    },
                 # -----------------------
                 'model_name': [
                            'distilbert/distilbert-base-uncased-finetuned-sst-2-english',
                           #  'michellejieli/emotion_text_classifier',
                           #  'cardiffnlp/twitter-xlm-roberta-base-sentiment',
                           #  'celine98/canine-s-finetuned-sst2',
                           #  'lxyuan/distilbert-base-multilingual-cased-sentiments-student',
                           #  'michelecafagna26/t5-base-finetuned-sst2-sentiment',
                           # 'nlptown/bert-base-multilingual-uncased-sentiment',
                           # 'ProsusAI/finbert',
                           # 'arpanghoshal/EmoRoBERTa',
                           # 'camembert-base'
                           # 'cardiffnlp/twitter-roberta-base-irony',
                           # 'cardiffnlp/twitter-roberta-base-sentiment-latest',
                           # 'ctrl',
                           # 'distilroberta-base',
                           # 'flaubert/flaubert_base_cased',
                           # 'j-hartmann/emotion-english-distilroberta-base',
                           # 'joeddav/distilbert-base-uncased-go-emotions-student',
                           # 'lxyuan/distilbert-base-multilingual-cased-sentiments-student',
                           # 'nlptown/bert-base-multilingual-uncased-sentiment',
                           # 'papluca/xlm-roberta-base-language-detection',
                           # 'roberta-base',
                           # 'xlnet-base-cased',
                           # 'facebook/tart-full-flan-t5-xl',
                           # 'lytang/MiniCheck-Flan-T5-Large',
                           # 'microsoft/phi-2',
                           # 'meta-llama/Meta-Llama-3-8B',
                           # 'lightblue/suzume-llama-3-8B-multilingual',
                           # 'google/gemma-2b',
                           # 'mistralai/Mistral-7B-v0.1',
                           # 'tiiuae/falcon-11B' ,
                           ], # Pre-trained model names from the Hugging Face hub used for fine-tuning
                 # --------------------------
                  'split': [
                           # (0.9, 0.1),
                           (0.8, 0.2),
                           # (0.7, 0.3),
                           # (0.6, 0.4),
                           # (0.5, 0.5),
                          ], # Divides the dataset into training, testing, (and optionally) validation sets. Use 'balanced' for equal class representation in the validation set. Examples: (90,10) -> split into train and test proportionally; (80, 10, 10) splits into train,test, validate proportionally.
                 'binary': False, # Indicates whether the task is binary (two classes) or multi-class classification.,
                 'balanced': False, # his way labels used for training are split evenly, fitting size to the lowest label count. n (equal to 80% of least represented label) will be taken from each label, rest will be used for test.
                 # --------------------------
                 'training_arguments': {
                     'num_train_epochs': 1, # Number of times the model sees the entire training dataset.
                     'per_device_train_batch_size': 8, # Number of samples processed in each training step (personally, 8/16 work best, 16 is faster, but you may find linear drop in inference speed during fine-tuning).
                     'per_device_eval_batch_size': 16, # Number of samples processed in each evaluation step.
                     # 'gradient_accumulation_steps': 4,
                     'gradient_checkpointing': True,
                     #-----------------------------
                     'save_total_limit': 2,
                     'load_best_model_at_end': True,
                     'save_strategy': 'steps', # Controls when to save model checkpoints ('steps', 'epoch' or 'no').
                     'metric_for_best_model': 'f1-score',
                     #-----------------------------
                     'evaluation_strategy': "steps",
                     'logging_steps': 20,
                     'max_steps': 40,
                     'fp16': False,
                     # 'use_cpu': False,
                     #-----------------------------
                     'learning_rate': [
                                        # 1e-6,
                                        5e-5,
                                        # 1e-5,
                                        # 1e-3,
                                        # 1e-2,
                                        ],
                     'lr_scheduler_type': [
                                       # "constant",
                                       # "constant_with_warmup",
                                         "linear",
                                       # "polynomial",
                                      ],
                     'warmup_ratio': [
                                      0.1,
                                      # 0.2,
                                      # 0.3
                                     ], #0.1
                     'max_grad_norm': [
                                       # 0.1,
                                       # 0.2,
                                       0.3
                                      ],  #0.3
                     'weight_decay': [
                                      0.001,
                                      # 0.01
                                     ], # 0.001
                 },
                 #-----------------------------
                 # 'callbacks': [EarlyStoppingCallback(early_stopping_patience=3)], #trainer not training args
                 'bnb_config': [
                                False,
                                # {bnb_4bit_compute_dtype=torch.bfloat16, 'load_in_4bit': True, 'bnb_4bit_quant_type': "nf4", 'bnb_4bit_use_double_quant': True}
                                 ],
                 'peft_config': [
                                False,
                                # {'r': 8,
                                # 'lora_alpha': 32,
                                # 'lora_dropout': 0.1,
                                # 'bias': "none",
                                # 'target_modules': ("q_proj", "v_proj",)
                                # 'target_modules': "all-linear"
                                # }
                                ],
                    }


# overwrite = {
#             'mistralai/Mistral-7B-v0.1' : {'training_params':{'per_device_train_batch_size': 8, 'per_device_eval_batch_size': 8, 'gradient_checkpointing': True,},
#                'peft_config' : LoraConfig(task_type=TaskType.SEQ_CLS,
#                                                     r=8,
#                                                     lora_alpha=32,
#                                                     lora_dropout=0.1,
#                                                     bias="none",
#                                                     target_modules='all-linear'
#                                                     # target_modules=[
#                                                          # "q_proj",
#                                                          # "v_proj",
#                                                             # ]
#                                                 )
#             }}

compute_dtype = getattr(torch, "float16")
print(compute_dtype)

# Controls whether to save logs during a process. When set to False, logging is disabled.
save_logs = True

train_params_looped = param_combinations_recursive(params_tested)
print(len(train_params_looped))
print(*train_params_looped, sep='\n')

torch.float16
1
{'data_path': 'data/PolarIs-Pathos.xlsx', 'wandb_init_params': {'project': 'test', 'group': 'binary'}, 'model_name': 'distilbert/distilbert-base-uncased-finetuned-sst-2-english', 'split': (0.8, 0.2), 'binary': False, 'balanced': False, 'training_arguments': {'num_train_epochs': 1, 'per_device_train_batch_size': 8, 'per_device_eval_batch_size': 16, 'gradient_checkpointing': True, 'save_total_limit': 2, 'load_best_model_at_end': True, 'save_strategy': 'steps', 'metric_for_best_model': 'f1-score', 'evaluation_strategy': 'steps', 'logging_steps': 20, 'max_steps': 40, 'fp16': False, 'learning_rate': 5e-05, 'lr_scheduler_type': 'linear', 'warmup_ratio': 0.1, 'max_grad_norm': 0.3, 'weight_decay': 0.001}, 'bnb_config': False, 'peft_config': False}


In [12]:
#run looped
for train_params in tqdm(train_params_looped):
    with open('run_params.pkl', 'wb') as file:
        pickle.dump(train_params, file)
        
    import runpy
    runpy.run_path(path_name='train.py')
    del runpy

    if torch.cuda.is_available():
        gc.collect()
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()
        torch.cuda.reset_accumulated_memory_stats()


if yeelight_notify:
    yeelight_eow_notification(dotenv_config['YEELIGHT_PORT'])

  0%|          | 0/1 [00:00<?, ?it/s]

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\Jakub\.cache\huggingface\token
Login successful


            text
label           
Negative    4106
No_pathos  10637
Positive     845


Casting to class labels:   0%|          | 0/15588 [00:00<?, ? examples/s]

C:\Users\Jakub\anaconda3\envs\EQILLM\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

C:\Users\Jakub\anaconda3\envs\EQILLM\Lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Precision,Recall,F1-score,Accuracy
20,0.765800,1.092010,0.452319,0.672547,0.540874,0.672547
40,1.041100,1.189414,0.452319,0.672547,0.540874,0.672547


C:\Users\Jakub\anaconda3\envs\EQILLM\Lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


15.3139 seconds used for training.
0.26 minutes used for training.
Peak reserved memory = 1.465 GB.
Peak reserved memory for training = 1.18 GB.
Peak reserved memory % of max memory = 14.65 %.
Peak reserved memory for training % of max memory = 11.8 %.


eval/accuracy,▁▁
eval/f1-score,▁▁
eval/loss,▁█
eval/precision,▁▁
eval/recall,▁▁
eval/runtime,█▁
eval/samples_per_second,▁█
eval/steps_per_second,▁█
train/epoch,▁▁███
train/global_step,▁▁█████
train/learning_rate,█▁


In [13]:
# #run looped
# train_params=train_params_looped[0]

# with open('run_params.pkl', 'wb') as file:
#     pickle.dump(train_params, file)

    
# # set run environment (local/colab), if colab move proper dir
# import os
# import json
# import pickle
# from pathlib import Path
# import torch, gc
# import wandb
# # from numba import cuda
# from csv import writer
# from datasets import Dataset, load_dataset
# from dotenv import load_dotenv, dotenv_values
# from huggingface_hub import login
# from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
# from transformers import AutoModelForSequenceClassification, Trainer, BitsAndBytesConfig, AutoTokenizer, DataCollatorWithPadding

# from eqillm import finetune, yeelight_eow_notification, param_combinations, load_PolarIs, split_ds, encode_labels, init_model, to_binary_classification, load_predefined_dataset

# dotenv_config = dotenv_values('env')
# # yeelight_notify = dotenv_config['YEELIGHT_NOTIFY'] if ('YEELIGHT_NOTIFY' in dotenv_config) and (colab) else False

# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# login(token=dotenv_config['HF_TOKEN'])

# with open('run_params.pkl', 'rb') as file: 
#     run_params = pickle.load(file)

# if run_params['bnb_config']!=False:
#     bnb_config = BitsAndBytesConfig(**run_params['bnb_config'])
# else:
#     bnb_config=False

# if run_params['peft_config']!=False:
#     peft_config = LoraConfig(**run_params['peft_config'])
# else:
#     peft_config=False
    
# wandb.init(name=run_params['model_name'], **run_params['wandb_init_params'])

# df = load_predefined_dataset(run_params['data_path'], run_params['binary'])
# print(df.groupby('label').count())
# ds = Dataset.from_pandas(df)
# ds = ds.class_encode_column('label')
# target_map = {i: ds.features["label"].str2int(i) for i in ds.features["label"].names}

# ds = split_ds(ds, train_size=run_params['split'][0])

# # with open('data.json', 'w', encoding='utf-8') as f:
# with open('quant_errors.csv', 'a+', newline='') as f_object:  
# #     writer_object = writer(f_object)
#     try:
#         model_name = run_params['model_name']
        
#         model, tokenizer, tokenized_datasets = init_model(run_params['model_name'], ds, target_map, bnb_config, peft_config)
        
#         # train_params = {k: run_params[k] for k in run_params if k not in ['data_path',
#         #                                                                   'model_name',
#         #                                                                   'split',
#         #                                                                   'binary',
#         #                                                                   'balanced',
#         #                                                                   'bnb_config',
#         #                                                                   'peft_config',
#         #                                                                   'wandb_init_params']}
#         # writer_object.writerow([model_name, 'works'])  
        
#         trainer = finetune(model, tokenizer, tokenized_datasets, ds, run_params['training_arguments'], target_map)
#     except Exception as exc:
#         print(exc)
#         wandb.log({'error': str(exc)})
#         wandb.finish(1)
#         # writer_object.writerow([model_name, exc])  

# # f_object.close()

# if torch.cuda.is_available():
#     gc.collect()
#     torch.cuda.empty_cache()
#     torch.cuda.reset_peak_memory_stats()
#     torch.cuda.reset_accumulated_memory_stats()


# if yeelight_notify:
#     yeelight_eow_notification(dotenv_config['YEELIGHT_PORT'])
    

In [5]:
# del trainer, model, tokenizer
# if torch.cuda.is_available():
#     gc.collect()
#     torch.cuda.empty_cache()
#     torch.cuda.reset_peak_memory_stats()
#     torch.cuda.reset_accumulated_memory_stats()

In [16]:
# df = load_predefined_dataset(run_params['data_path'], run_params['binary'])
# print(df.groupby('label').count())
# ds = Dataset.from_pandas(df)
# ds = ds.class_encode_column('label')
# target_map = {i: ds.features["label"].str2int(i) for i in ds.features["label"].names}

# # ds = split_ds(ds, train_size=run_params['split'][0])

# ds.train_test_split(train_size=0.3, seed=42, stratify_by_column='label')['train'].to_pandas().groupby(['label']).count()

,text
label,
0,72
1,43
